In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("./")

### Q1. Install the package

In [1]:
!mlflow --version

mlflow, version 2.3.2


### Q2. Download and preprocess the data

In [55]:
!pwd

/Users/rguarise/Documents/mlops-zoomcamp/homeworks/02-experiment-tracking


In [58]:
!python ./utils/preprocess_data.py --raw_data_path $HOME/Documents/mlops-zoomcamp/data/ --dest_path ./output

In [59]:
!ls -l ./output

total 17320
-rw-r--r--  1 rguarise  1010544492   153660 Jun  1 21:41 dv.pkl
-rw-r--r--  1 rguarise  1010544492  2632817 Jun  1 21:41 test.pkl
-rw-r--r--  1 rguarise  1010544492  2146163 Jun  1 21:41 train.pkl
-rw-r--r--  1 rguarise  1010544492  2336393 Jun  1 21:41 val.pkl


### Q3. Train a model with autolog

In [5]:
import utils.train as train
train.run_train("./output/")

2023/06/01 22:41:14 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'eea9dd79fe704d8b9b022810a3060c39', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
2023/06/01 22:41:27 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/rguarise/Library/Caches/pypoetry/virtualenvs/poetry_shell-XdBlMnFf-py3.8/lib/python3.8/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils."


### Q4. Tune model hyperparameters

In [2]:
import utils.hpo as hpo
hpo.run_optimization("./output/",10)

[I 2023-06-01 23:28:54,663] A new study created in memory with name: no-name-b64a1f27-0c28-4361-8a2a-b58f9b82fff8
2023/06/01 23:29:08 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/rguarise/Library/Caches/pypoetry/virtualenvs/poetry_shell-XdBlMnFf-py3.8/lib/python3.8/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils."
[I 2023-06-01 23:29:09,455] Trial 0 finished with value: 2.451379690825458 and parameters: {'n_estimators': 25, 'max_depth': 20, 'min_samples_split': 8, 'min_samples_leaf': 3}. Best is trial 0 with value: 2.451379690825458.
[I 2023-06-01 23:29:19,114] Trial 1 finished with value: 2.4667366020368333 and parameters: {'n_estimators': 16, 'max_depth': 4, 'min_samples_split': 2, 'min_samples_leaf': 4}. Best is trial 0 with value: 2.451379690825458.
[I 2023-06-01 23:29:31,683] Trial 2 finished with value: 2.449827329704216 and parameters: {'n_estimators': 34, 'max_depth': 15, 'min_samples_spli

### Q5. Promote the best model to the model registry

In [2]:
import utils.register_model as register_model
register_model.run_register_model("./output/",1)

2023/06/01 23:35:27 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/rguarise/Library/Caches/pypoetry/virtualenvs/poetry_shell-XdBlMnFf-py3.8/lib/python3.8/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils."
Registered model 'best_model' already exists. Creating a new version of this model...
2023/06/01 23:35:27 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: best_model, version 3


test_rmse: 2.2854691906481364


Created version '3' of model 'best_model'.
